In [1]:
import cobra
from cobra import Reaction
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import ast

# Save JSON model with pythonified IDs and subsystem information

In [2]:
M = cobra.io.load_matlab_model('Recon3DModel_301.mat') # takes several minutes

# fix reaction and metabolite IDs
for rxn in M.reactions:
    rxn.id = rxn.id.replace('[','_').replace(']','')

M.repair()

for m in M.metabolites:
    m.id = m.id.replace('[','_').replace(']','')

M.repair()

# add ATPM reaction
ATPM = Reaction('ATPM')
M.add_reaction(ATPM)
M.reactions.ATPM.lower_bound = 0
M.reactions.ATPM.add_metabolites({'atp_c':-1.0,'adp_c':1.0,'pi_c':1.0})
M.repair()

# set objective to full biomass
M.reactions.biomass_maintenance.objective_coefficient = 0
M.reactions.biomass_reaction.objective_coefficient = 1

# Add subsystem information
rxn_ss = pd.read_csv('./subsystem_data/Recon3_rxn_subsystem.csv',index_col=0).set_index('rxn ID')

for rxn in rxn_ss.index:
    rxn_updated_id = rxn.replace('[','_').replace(']','')
    
    ss = ast.literal_eval(rxn_ss.loc[rxn]['subSystem'])
    
    M.reactions.get_by_id(rxn_updated_id).subsystem = ss
    
# save
cobra.io.save_json_model(M, 'Recon3DModel_301_pythonified.json')

# Reduce medium + exits
Set up a simple medium of glucose, oxygen, essential amino acids and a couple of other metabolites. Block all sinks and DM reactions. 

In [9]:
model = M.copy()

# Block sink and demand reactions
for rxn in model.reactions:
    if 'sink_' in rxn.id or 'DM_' in rxn.id:
        rxn.upper_bound = 0
        rxn.lower_bound = 0
        
# set very simple medium that allows growth
for rxn in model.reactions:
    if 'EX_' in rxn.id:
        rxn.upper_bound = 0 # block all outward
        rxn.lower_bound = 0
        
for rxn in ['EX_h2o_e','EX_h_e','EX_co2_e','EX_nh4_e','EX_pi_e','EX_hco3_e','EX_so4_e','EX_o2_e']:
    model.reactions.get_by_id(rxn).lower_bound = - 1000

for rxn in ['EX_his_L_e','EX_ile_L_e','EX_leu_L_e','EX_lys_L_e','EX_met_L_e','EX_phe_L_e','EX_thr_L_e','EX_trp_L_e','EX_val_L_e',
            'EX_ps_hs_e', 'EX_glu_L_e', 'EX_gln_L_e' # phsophatidylserine
           ]:
    model.reactions.get_by_id(rxn).lower_bound = -0.1
    
model.reactions.EX_glc_D_e.lower_bound = -1
        
for rxn in ['EX_h2o_e','EX_h_e','EX_nh4_e','EX_pi_e','EX_so4_e',
           'EX_co2_e','EX_lac_D_e','EX_lac_L_e',
           'EX_urea_e','EX_acac_e','EX_bhb_e','EX_ppa_e','EX_cyst_L_e' ]: # 
    model.reactions.get_by_id(rxn).upper_bound = 1000
        
cobra.io.save_json_model(model, 'Recon3DModel_301_simple_medium.json')

sol = cobra.flux_analysis.pfba(model)
print('After changes',sol.fluxes['biomass_reaction'])
model.summary()

After changes 0.16888639687627716
IN FLUXES        OUT FLUXES        OBJECTIVES
---------------  ----------------  ----------------------
o2_e     2.42    h2o_e    2.65     biomass_reac...  0.169
glc_D_e  1       co2_e    2.19
nh4_e    0.241   lac_D_e  0.658
pi_e     0.162   bhb_e    0.365
gln_L_e  0.1     ppa_e    0.00787
glu_L_e  0.1
his_L_e  0.1
leu_L_e  0.1
lys_L_e  0.1
trp_L_e  0.1
val_L_e  0.1
phe_L_e  0.0708
thr_L_e  0.0528
ile_L_e  0.0483
ps_hs_e  0.0477
met_L_e  0.0337


In [10]:
# keep M unchanged
M = model.copy()
model = M.copy()

# Check oxygen dependence

In [11]:
model.reactions.EX_o2_e.lower_bound = 0
print(model.optimize())

# model.metabolites.get_by_id('o2_c').summary()

<Solution -0.000 at 0x13ad2f550>


## How is ATP generated? 
### When maximizing ATPM using only glucose and o2

In [15]:
model = M.copy()

for rxn in ['EX_his_L_e','EX_ile_L_e','EX_leu_L_e','EX_lys_L_e','EX_met_L_e','EX_phe_L_e','EX_thr_L_e','EX_trp_L_e','EX_val_L_e',
           'EX_glu_L_e','EX_gln_L_e']:
    model.reactions.get_by_id(rxn).lower_bound = 0

model.objective = model.reactions.ATPM
model.optimize()

model.summary()
print()
model.metabolites.atp_c.summary()
print()
model.metabolites.atp_m.summary()

IN FLUXES    OUT FLUXES    OBJECTIVES
-----------  ------------  ------------
h_e      32  h2o_e  38     ATPM  32
o2_e      6  co2_e   6
glc_D_e   1

PRODUCING REACTIONS -- Adenosine Triphosphate (atp_c)
-----------------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  ---------------------------------------
94%      58  ATPtm     adp_c + atp_m --> adp_m + atp_c
3%        2  PGK       3pg_c + atp_c <=> 13dpg_c + adp_c
3%        2  PYK       adp_c + h_c + pep_c --> atp_c + pyr_c

CONSUMING REACTIONS -- Adenosine Triphosphate (atp_c)
-----------------------------------------------------
%      FLUX  RXN ID    REACTION
---  ------  --------  ---------------------------------------
52%      32  ATPM      atp_c --> adp_c + pi_c
45%      28  r0377     atp_c + dcyt_c --> adp_c + dcmp_c + h_c
2%        1  HEX7      atp_c + fru_c --> adp_c + f6p_c + h_c
2%        1  PFK       atp_c + f6p_c --> adp_c + fdp_c + h_c

PRODUCING REACTIONS -- Adenosine Triph

### When maximizing biomass

In [16]:
model = M.copy()

model.optimize()

model.summary()
print()
model.metabolites.atp_c.summary()
print()
model.metabolites.atp_m.summary()

IN FLUXES        OUT FLUXES      OBJECTIVES
---------------  --------------  ----------------------
o2_e     2.89    h2o_e   4.34    biomass_reac...  0.169
glc_D_e  1       co2_e   2.68
nh4_e    0.28    acac_e  0.617
pi_e     0.163   ppa_e   0.0556
gln_L_e  0.1     so4_e   0.0477
glu_L_e  0.1
his_L_e  0.1
leu_L_e  0.1
lys_L_e  0.1
thr_L_e  0.1
val_L_e  0.1
met_L_e  0.0815
phe_L_e  0.0708
ile_L_e  0.0483
ps_hs_e  0.0463
trp_L_e  0.034
hco3_e   0.0194

PRODUCING REACTIONS -- Adenosine Triphosphate (atp_c)
-----------------------------------------------------
%      FLUX  RXN ID      REACTION
---  ------  ----------  --------------------------------------------------
52%  3.9     ATPtm       adp_c + atp_m --> adp_m + atp_c
25%  1.83    PGK         3pg_c + atp_c <=> 13dpg_c + adp_c
23%  1.71    PYK         adp_c + h_c + pep_c --> atp_c + pyr_c
0%   0.0189  NDPK6       atp_c + dudp_c <=> adp_c + dutp_c

CONSUMING REACTIONS -- Adenosine Triphosphate (atp_c)
----------------------------------

## How does the model behave under oxygen limitation? 

In [17]:
model = M.copy()

model.reactions.EX_o2_e.lower_bound = -0.1
sol = cobra.flux_analysis.pfba(model)

model.summary()
print()
model.metabolites.atp_c.summary()
print()
model.metabolites.atp_m.summary()

IN FLUXES         OUT FLUXES        OBJECTIVES
----------------  ----------------  ----------------------
glc_D_e  1        lac_D_e  1.67     biomass_reac...  0.084
hco3_e   0.46     h_e      1.31
his_L_e  0.1      h2o_e    0.65
o2_e     0.1      co2_e    0.638
gln_L_e  0.0979   bhb_e    0.0573
pi_e     0.0812   ppa_e    0.00391
lys_L_e  0.0497
leu_L_e  0.0458
nh4_e    0.0445
phe_L_e  0.0352
val_L_e  0.0296
thr_L_e  0.0263
ile_L_e  0.024
ps_hs_e  0.023
met_L_e  0.0168
trp_L_e  0.00112

PRODUCING REACTIONS -- Adenosine Triphosphate (atp_c)
-----------------------------------------------------
%      FLUX  RXN ID      REACTION
---  ------  ----------  --------------------------------------------------
61%  1.92    PGK         3pg_c + atp_c <=> 13dpg_c + adp_c
28%  0.893   PYK         adp_c + h_c + pep_c --> atp_c + pyr_c
11%  0.346   ATPtm       adp_c + atp_m --> adp_m + atp_c

CONSUMING REACTIONS -- Adenosine Triphosphate (atp_c)
-----------------------------------------------------
%  